In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


In [2]:
# Now as always we get the data we stored in the disk.
cleaned_dataset_path = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Udacity/DataPreparation/dataset_cleaned.p'

with open(cleaned_dataset_path, 'rb') as f:
    fnl_dataset = pickle.load(f)
    training_dataset = (fnl_dataset['training_dataset'])
    training_labels = (fnl_dataset['training_labels'])
    test_dataset = (fnl_dataset['test_dataset'])
    test_labels = (fnl_dataset['test_labels'])
    crossvalid_dataset = (fnl_dataset['crossvalid_dataset'])
    crossvalid_labels = (fnl_dataset['crossvalid_labels'])
    
print('Training:', training_dataset.shape, training_labels.shape)
print('Cross Validation:', crossvalid_dataset.shape, crossvalid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Cross Validation: (9810, 28, 28) (9810,)
Testing: (7709, 28, 28) (7709,)


In [60]:
# Mini-Batch Creation
no_of_batches = 200
batch_indices = []
def batch_indice_creator(training_labels, no_of_batches):
    """Create batch with random samples and return appropriate format"""
    label_rand_arr = []
    no_of_labels = len(np.unique(training_labels))
    
    if len(training_labels)/no_of_batches < no_of_labels:
        raise Exception ("Every batch should contain atleast 1 example from all the differnt class labels")
    else:
        for unq_labels in np.unique(training_labels):
            arr = np.array(np.where(training_labels==unq_labels)[0])
            print ('Total number of label %d is :'%unq_labels, len(arr))
            np.random.shuffle(arr)
    #         print ('the random array is: ', np.reshape(arr, (len(arr),1)))
            if np.any(label_rand_arr):
                label_rand_arr = np.hstack((label_rand_arr, np.reshape(arr, (len(arr),1))))
            else:
                label_rand_arr = np.reshape(arr, (len(arr),1))
        
        no_of_batches_new = no_of_batches
        if label_rand_arr.shape[0]%no_of_batches!=0:
            no_of_batches_new = no_of_batches + label_rand_arr.shape[0]%no_of_batches
            print ('Provided no_of_batches doesnt distribute the labels equally, ne no_of_batches is: ', no_of_batches_new) 
        batch_indices = np.array_split(label_rand_arr.flatten(), no_of_batches_new)
        batch_size = len(batch_indices[0])
    return batch_indices, batch_size


training_batch_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/'        
batch_training_dict = {}
def batch_store(training_dataset, training_labels, batch_indices):
    for no, indices in enumerate(batch_indices):
        #print (training_labels[indices])
        try:
            f = open(training_batch_dir+'batch'+str(no)+'.pickle', 'wb')
            batch = {
                'batch_train_dataset': training_dataset[indices],
                'batch_train_labels': training_labels[indices],
            }
            pickle.dump(batch, f, pickle.HIGHEST_PROTOCOL)
            f.close()
        except Exception as e:
            print('Unable to save data to', complete_dataset_path, ':', e)
            raise
            
batch_indices, batch_size = batch_indice_creator(training_labels, no_of_batches)  #print ([arr.shape for arr in batch_indices])
#batch_store(training_dataset, training_labels, batch_indices)

print ('the batch size is:',batch_size )


Total number of label 0 is : 20000
Total number of label 1 is : 20000
Total number of label 2 is : 20000
Total number of label 3 is : 20000
Total number of label 4 is : 20000
Total number of label 5 is : 20000
Total number of label 6 is : 20000
Total number of label 7 is : 20000
Total number of label 8 is : 20000
Total number of label 9 is : 20000
the batch size is: 1000


In [46]:
image_size = 28
no_of_labels = 10
no_inp_unit = image_size * image_size

# tensor flow takes the labels input as binary code, where Alphabet A whose binary value is 0 will turn to a array
# with elements [1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0] and B becomes [0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def reshape_data(dataset, labels, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape(sample_size, no_inp_unit) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(no_of_labels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape(len(dataset), no_inp_unit) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(no_of_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

# We just reshape the image so that 1 image defines a row

crossvalid_dataset_, crossvalid_labels_ = reshape_data(crossvalid_dataset, crossvalid_labels)
test_dataset_, test_labels_ = reshape_data(test_dataset, test_labels)
print('Cross Validation set', crossvalid_dataset_.shape, crossvalid_labels_.shape)
print('Test set', test_dataset_.shape, test_labels_.shape)

Cross Validation set (9810, 784) (9810, 10)
Test set (7709, 784) (7709, 10)


In [79]:
# Building the Tensor Flow Graph
learning_rate = 0.5
momentum = 0.9
no_hid_unit = 200
no_output_unit = no_of_labels

graph = tf.Graph()
with graph.as_default():
    seed = 128
    rng = np.random.RandomState(seed)
    tf_training_dataset = tf.placeholder(tf.float32, shape=(batch_size, no_inp_unit))
    tf_training_labels = tf.placeholder(tf.float32, shape=(batch_size, no_of_labels))
    tf_crossvalid_dataset = tf.constant(crossvalid_dataset_)
    tf_crossvalid_labels = tf.constant(crossvalid_labels_)
    tf_test_dataset = tf.constant(test_dataset_)
    tf_test_labels = tf.constant(test_labels_)


    weights = {
        'input_to_hid_wghts': tf.Variable(tf.random_normal([no_inp_unit, no_hid_unit], seed=seed)),
        'hid_to_output_wghts': tf.Variable(tf.random_normal([no_hid_unit, no_output_unit], seed=seed))
    }

    biases = {
        'hid_bias' : tf.Variable(tf.zeros([no_hid_unit])),
        'output_bias' : tf.Variable(tf.zeros([no_output_unit]))
    }

    ###### Forward Propagate ######
    # Hidden Layer
    input_to_hid_layer = tf.matmul(tf_training_dataset, weights['input_to_hid_wghts']) + biases['hid_bias']
    hid_layer_state = tf.sigmoid(input_to_hid_layer, name=None)

    # Output Layer
    hid_to_output_layer = tf.matmul(hid_layer_state, weights['hid_to_output_wghts']) + biases['output_bias']
    output_layer_state = tf.nn.softmax(hid_to_output_layer, name=None)
    error_derivative = tf.sub(output_layer_state, tf_training_labels, name=None)  # -(y - y_hat) = (y_hat - y)
    # The above three lines of code can also be combined into one line as below.
    loss_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hid_to_output_layer, tf_training_labels))

    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_CE)
    
    optimizer = tf.train.MomentumOptimizer(learning_rate, 
                                            momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(loss_CE)
    # Training Prediction
    training_prediction = output_layer_state
    # Cross-validation Prediction
    cvd_hid_layer_state = tf.sigmoid(tf.matmul(tf_crossvalid_dataset, weights['input_to_hid_wghts']) + biases['hid_bias'], name=None)
    crossvalid_prediction = tf.nn.softmax(tf.matmul(cvd_hid_layer_state, weights['hid_to_output_wghts']) + biases['output_bias'])
    # Test Data Prediction
    tst_hid_layer_state = tf.sigmoid(tf.matmul(tf_test_dataset, weights['input_to_hid_wghts']) + biases['hid_bias'], name=None)
    test_prediction = tf.nn.softmax(tf.matmul(tst_hid_layer_state, weights['hid_to_output_wghts']) + biases['output_bias'])
    


In [80]:
epochs = 10

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("All variable Initialized successfully")
    for epoch in range(epochs):
        # Retreive mini-batches
        for no in range(no_of_batches):
            #print (no)
            with open(training_batch_dir+'batch'+str(no)+'.pickle', 'rb') as f:
                batch_dataset = pickle.load(f)
                batch_data = (batch_dataset['batch_train_dataset'])
                batch_labels = (batch_dataset['batch_train_labels'])
                batch_data, batch_labels = reshape_data(batch_data, batch_labels)
                
                feed_dict = {tf_training_dataset : batch_data, tf_training_labels : batch_labels}
                _, l, predictions = session.run([optimizer, loss_CE, training_prediction], feed_dict=feed_dict)
        
        if (epoch % 2 == 0) or epoch == epochs-1:
            print("Minibatch loss at epoch %d: %f" % (epoch, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(crossvalid_prediction.eval(), crossvalid_labels_))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_))

All variable Initialized successfully
Minibatch loss at epoch 0: 0.771911
Minibatch accuracy: 79.5%
Validation accuracy: 59.2%
Minibatch loss at epoch 2: 0.602523
Minibatch accuracy: 82.7%
Validation accuracy: 64.0%
Minibatch loss at epoch 4: 0.565977
Minibatch accuracy: 84.3%
Validation accuracy: 65.2%
Minibatch loss at epoch 6: 0.539594
Minibatch accuracy: 84.6%
Validation accuracy: 65.7%
Minibatch loss at epoch 8: 0.517757
Minibatch accuracy: 84.9%
Validation accuracy: 66.0%
Minibatch loss at epoch 9: 0.508369
Minibatch accuracy: 85.3%
Validation accuracy: 66.1%
Test accuracy: 88.7%


In [ ]:
# Note :
# For the momentum optimizer using Nesterov's method we get a very good accuracy rate
# Minibatch accuracy: 85.3%  --> training accuracy
# Validation accuracy: 66.1%
# Test accuracy: 88.7%

In [ ]:
# Normally Tensor flow does the Back-propogation automatically after you build the graph and run the 
# session. But by any chance if you want to implement your own backpropogation. then do the below.


###########  Practise  #############

# graph = tf.Graph()
# with graph.as_default():

#     seed = 128
#     rng = np.random.RandomState(seed)
#     tf_training_dataset = tf.placeholder(tf.float32, shape=(batch_size, no_inp_unit))
#     tf_training_labels = tf.placeholder(tf.float32, shape=(batch_size, no_of_labels))
# #     tf_training_dataset = tf.constant(training_dataset_)
# #     tf_training_labels = tf.constant(training_labels_)

#     weights = {
#         'input_to_hid_wghts': tf.Variable(tf.random_normal([no_inp_unit, no_hid_unit], seed=seed)),
#         'hid_to_output_wghts': tf.Variable(tf.random_normal([no_hid_unit, no_output_unit], seed=seed)),
#         'input_to_hid_wghts_delta': tf.Variable(tf.zeros([no_inp_unit, no_hid_unit])),
#         'hid_to_output_wghts_delta': tf.Variable(tf.zeros([no_hid_unit, no_output_unit])),
#     }

#     biases = {
#         'hid_bias' : tf.Variable(tf.zeros([no_hid_unit])),
#         'output_bias' : tf.Variable(tf.zeros([no_output_unit])),
#         'hid_bias_delta' : tf.Variable(tf.zeros([no_hid_unit])),
#         'output_bias_delta' : tf.Variable(tf.zeros([no_output_unit]))
#     }


#     ###### Forward Propagate ######
#     # Hidden Layer
#     input_to_hid_layer = tf.add(tf.matmul(tf_training_dataset, weights['input_to_hid_wghts']), biases['hid_bias'])
#     hid_layer_state = tf.sigmoid(inputs_to_hid, name=None)

#     # Output Layer
#     hid_to_output_layer = tf.add(tf.matmul(hid_layer_state, weights['hid_to_output_wghts']), biases['output_bias'])
#     output_layer_state = tf.nn.softmax(hid_to_output_layer, name=None)
#     error_derivative = tf.sub(output_layer_state, tf_training_labels, name=None)  # -(y - y_hat) = (y_hat - y)
#     # The above three lines of code can also be combined into one line as below.
#     loss_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hid_to_output_layer, tf_training_labels))


#     ####### Back Propagate #######
#     # Output_Layer
#     hid_to_output_wghts_gradient =  tf.matmul(tf.transpose(hid_layer_state), error_derivative)
#     output_bias_gradient =  tf.reduce_sum(error_derivative, 0)  # sum all the row_value for each column 

#     # Hidden_Layer
#     back_propagate_deriv = tf.mul(tf.transpose(tf.matmul(weights['hid_to_output_wghts'], tf.transpose(error_derivative))),
#                                   tf.mul(hid_layer_state, 1.0-hid_layer_state)
#                                  )
#     input_to_hid_wghts_gradient = tf.matmul(tf.transpose(tf_training_dataset), back_propagate_deriv)
#     hid_bias_gradient = tf.reduce_sum(back_propagate_deriv, 0)

#     ####### Weight Update  #######
#     # Update Weights
#     weights['input_to_hid_wghts_delta'] = tf.mul(momentum, weights['input_to_hid_wghts_delta']) \
#                                 + tf.div(input_to_hid_wghts_gradient, batch_size)
#     weights['input_to_hid_wghts'] = weights['input_to_hid_wghts'] - tf.mul(learning_rate,weights['input_to_hid_wghts_delta'])

#     weights['hid_to_output_wghts_delta'] = tf.mul(momentum, weights['hid_to_output_wghts_delta']) \
#                                 + tf.div(hid_to_output_wghts_gradient, batch_size)
#     weights['hid_to_output_wghts'] = weights['hid_to_output_wghts'] - tf.mul(learning_rate,weights['hid_to_output_wghts_delta'])


#     ####### Bias Update  #######
#     biases['hid_bias_delta'] = tf.mul(momentum, biases['hid_bias_delta']) + tf.div(hid_bias_gradient, batch_size)
#     biases['hid_bias'] = biases['hid_bias'] - tf.mul(learning_rate,biases['hid_bias_delta'])

#     biases['output_bias_delta'] = tf.mul(momentum, biases['output_bias_delta']) + tf.div(output_bias_gradient, batch_size)
#     biases['output_bias'] = biases['output_bias'] - tf.mul(learning_rate,biases['output_bias_delta'])

    

In [ ]:
# And then We can print and look at the data:

# with tf.Session() as sess:
#     sess.run(init)
#     for i in range(2):
#         print ('Forward Pass')
#         print ('The input_to_hid_layer is: ', [w for no, w in enumerate(sess.run(input_to_hid_layer))])
#         print ('')
#         print ('The hid_layer_state is: ', [w for no, w in enumerate(sess.run(hid_layer_state))])
#         print ('')
#         print ('The hid_to_output_layer is: ', [w for no, w in enumerate(sess.run(hid_to_output_layer))])
#         print ('')
#         print ('The output_layer_state is: ', [w for no, w in enumerate(sess.run(output_layer_state))])
#         print ('')
#         print ('The error_derivative is: ', [w for no, w in enumerate(sess.run(error_derivative))])
#         print ('')
#         print ('The Loss value: ', sess.run(loss_CE))
#         print ('')
#         print ('')

#         print ('Back Propagate')
#         print ('The hid_to_output_wghts_gradient is: ', [w for no, w in enumerate(sess.run(hid_to_output_wghts_gradient))])
#         print ('')
#         print ('The hid_to_output_bias_gradient is: ', [w for no, w in enumerate(sess.run(hid_to_output_bias_gradient))])
#         print ('')
#         print ('The back_propagate_deriv is: ', [w for no, w in enumerate(sess.run(back_propagate_deriv))])
#         print ('')
#         print ('The input_to_hid_wghts_gradient is: ', [w for no, w in enumerate(sess.run(input_to_hid_wghts_gradient))])
#         print ('')
#         print ('')

#         print ('Weight Update')
#         print ('The input_to_hid_wghts_delta is: ', [w for no, w in enumerate(sess.run(weights['input_to_hid_wghts_delta']))])
#         print ('')
#         print ('The new input_to_hid_wghts is: ', [w for no, w in enumerate(sess.run(weights['input_to_hid_wghts']))])
#         print ('')
#         print ('The hid_to_output_wghts_delta is: ', [w for no, w in enumerate(sess.run(weights['hid_to_output_wghts_delta']))])
#         print ('')
#         print ('The new hid_to_output_wghts is: ', [w for no, w in enumerate(sess.run(weights['hid_to_output_wghts']))])
#         print ('')
#         print ('')

#         print ('Bias Update')
#         print ('The hid_bias_delta is: ', [w for no, w in enumerate(sess.run(biases['hid_bias_delta']))])
#         print ('')
#         print ('The new hid_bias is: ', [w for no, w in enumerate(sess.run(biases['hid_bias']))])
#         print ('')
#         print ('The output_bias_delta is: ', [w for no, w in enumerate(sess.run(biases['output_bias_delta']))])
#         print ('')
#         print ('The new output_bias is: ', [w for no, w in enumerate(sess.run(biases['output_bias']))])
#         print ('')
#         print ('')
#         print ('')
#         print ('')
#         print ('')
#         print ('')
    